In [1]:
Network = require 'Network'
json = require 'json'

-- Options can be overrided on command line run.
cmd = torch.CmdLine()
-- cmd:option('-loadModel', false, 'Load previously saved model')
cmd:option('-saveModel', true, 'Save model after training/testing')
cmd:option('-modelName', 'VideoSegmentation', 'Name of class containing architecture')
cmd:option('-nGPU', 1, 'Number of GPUs, set -1 to use CPU')
cmd:option('-trainingSetLMDBPath', './prepare_datasets/lmdb/train/', 'Path to LMDB training dataset')
cmd:option('-validationSetLMDBPath', './prepare_datasets/lmdb/test/', 'Path to LMDB test dataset')
cmd:option('-logsTrainPath', './logs/TrainingLoss/', ' Path to save Training logs')
cmd:option('-logsValidationPath', './logs/ValidationScores/', ' Path to save Validation logs')
cmd:option('-saveModelInTraining', false, 'save model periodically through training')
cmd:option('-modelTrainingPath', './models/', ' Path to save periodic training models')
cmd:option('-saveModelIterations', 50, 'When to save model through training')
cmd:option('-modelPath', 'video_segmentation.t7', 'Path of final model to save/load')
-- cmd:option('-dictionaryPath', './equ_dictionary', ' File containing the dictionary to use')
cmd:option('-epochs', 70, 'Number of epochs for training')
cmd:option('-learningRate', 3e-4, ' Training learning rate')
cmd:option('-learningRateAnnealing', 1.1, 'Factor to anneal lr every epoch')
-- cmd:option('-maxNorm', 400, 'Max norm used to normalize gradients')
cmd:option('-momentum', 0.90, 'Momentum for SGD')
cmd:option('-batchSize', 10, 'Batch size in training')
cmd:option('-validationBatchSize', 10, 'Batch size for validation')

opt = cmd:parse("")

net_opt = json.load('params.json')

for k,v in pairs(net_opt) do opt[k] = v end

--Parameters for the stochastic gradient descent (using the optim library).
optimParams = {
    learningRate = opt.learningRate,
    learningRateAnnealing = opt.learningRateAnnealing,
    momentum = opt.momentum,
    dampening = 0,
    nesterov = true
}

--Create and train the network based on the parameters and training data.
Network:init(opt)

In [2]:
inds = Network.indexer:nextIndices()

In [3]:
loader = Loader(Network.trainingSetLMDBPath)

In [4]:
clipBuf, labelBuf = loader:nextBatch(inds)

In [5]:
m1 = Network.model:get(1)

In [6]:
m2 = Network.model:get(2)

In [7]:
m3 = Network.model:get(3)

In [8]:
d1 = m1:forward(clipBuf:cuda())

In [9]:
d2 = m2:forward(d1)

In [10]:
d3 = m3:forward(d2)

In [11]:
criterion = nn.CrossEntropyCriterion():cuda()

In [12]:
criterion:forward(d3, labelBuf)

1.12573158741	
